## Customer Churn Preds .. Imbalaced Data

### Scenario: Imagine you have trained and fine-tuned your model and used it to make predictions

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import classification_report, confusion_matrix

## Load Data
### Customers that belong to class 0 are normal
### Customers that belong to class 1 require follow up


In [ ]:
# Load data
df = pd.read_csv('churn_preds.csv')

In [ ]:
df.head()

In [ ]:
df['Actual'].value_counts()

In [ ]:
df.tail()

## Binary Classifier Metrics

In [ ]:
# Reference: https://scikit-learn.org/stable/modules/model_evaluation.html
# Explicitly stating labels. Pass=1, Fail=0
def true_positive(y_true, y_pred): 
    return confusion_matrix(y_true, y_pred,labels=[1,0])[0, 0]

def true_negative(y_true, y_pred): 
    return confusion_matrix(y_true,y_pred,labels=[1,0])[1, 1]

def false_positive(y_true, y_pred): 
    return confusion_matrix(y_true, y_pred,labels=[1,0])[1, 0]

def false_negative(y_true, y_pred): 
    return confusion_matrix(y_true, y_pred,labels=[1,0])[0, 1]

In [ ]:
# Compute Binary Classifier Metrics
# Returns a dictionary {"MetricName":Value,...}

def binary_classifier_metrics(y_true, y_pred):
    metrics = {}

    # References: 
    #  https://docs.aws.amazon.com/machine-learning/latest/dg/binary-classification.html
    #  https://en.wikipedia.org/wiki/Confusion_matrix
    
    # Definition:
    # true positive = tp = how many samples were correctly classified as positive (count)
    # true negative = tn = how many samples were correctly classified as negative (count)
    # false positive = fp = how many negative samples were mis-classified as positive (count)
    # false_negative = fn = how many positive samples were mis-classified as negative (count)
    
    # positive = number of positive samples (count)
    #          = true positive + false negative
    # negative = number of negative samples (count)
    #          = true negative + false positive
    
    tp = true_positive(y_true, y_pred)
    tn = true_negative(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)
    
    positive = tp + fn
    negative = tn + fp
    
    metrics['TruePositive'] = tp
    metrics['TrueNegative'] = tn
    metrics['FalsePositive'] = fp
    metrics['FalseNegative'] = fn
    
    metrics['Positive'] = positive
    metrics['Negative'] = negative
    
    # True Positive Rate (TPR, Recall) = true positive/positive
    # How many positives were correctly classified? (fraction)
    # Recall value closer to 1 is better. closer to 0 is worse
    if tp == 0:
        recall = 0
    else:
        recall = tp/positive
        
    metrics['Recall'] = recall
    
    # True Negative Rate = True Negative/negative
    # How many negatives were correctly classified? (fraction)
    # True Negative Rate value closer to 1 is better. closer to 0 is worse
    if tn == 0:
        tnr = 0
    else:
        tnr = tn/(negative)
    metrics['TrueNegativeRate'] = tnr
    
    # Precision = True Positive/(True Positive + False Positive)
    # How many positives classified by the algorithm are really positives? (fraction)
    # Precision value closer to 1 is better. closer to 0 is worse
    if tp == 0:
        precision = 0
    else:
        precision = tp/(tp + fp)
    metrics['Precision'] = precision
    
    # Accuracy = (True Positive + True Negative)/(total positive + total negative)
    # How many positives and negatives were correctly classified? (fraction)
    # Accuracy value closer to 1 is better. closer to 0 is worse
    accuracy = (tp + tn)/(positive + negative)
    metrics['Accuracy'] = accuracy
    
    # False Positive Rate (FPR, False Alarm) = False Positive/(total negative)
    # How many negatives were mis-classified as positives (fraction)
    # False Positive Rate value closer to 0 is better. closer to 1 is worse
    if fp == 0:
        fpr = 0
    else:
        fpr = fp/(negative)
    metrics['FalsePositiveRate'] = fpr
    
    # False Negative Rate (FNR, Misses) = False Negative/(total Positive)
    # How many positives were mis-classified as negative (fraction)
    # False Negative Rate value closer to 0 is better. closer to 1 is worse
    fnr = fn/(positive)
    metrics['FalseNegativeRate'] = fnr
    
    # F1 Score = harmonic mean of Precision and Recall
    # F1 Score closer to 1 is better. Closer to 0 is worse.
    if precision == 0 or recall == 0:
        f1 = 0
    else:        
        f1 = 2*precision*recall/(precision+recall)

    metrics['F1'] = f1
    
    return metrics

In [ ]:
# Reference: 
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
    #    print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
df.columns

## Remember: Class 1 is the class of interest (customers that require follow up)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(df['Actual'], df['Predicted'],labels=[1,0])

In [ ]:
# Plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Followup','Normal'],
                      title='Confusion Matrix')

* Among the positive customers, the model predicted 106 of them correctly and mis-classified 53 as normal
* Among the negative customers, the model predicted 928 of them correctly and mis-classified 13 as positive

In [ ]:
# Plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Followup','Normal'],
                      title='Confusion Matrix - Fraction', normalize=True)

When viewed as a percentage, the model missed 33% of positive customers and 1% are false alarms

In [ ]:
metrics = [binary_classifier_metrics(df['Actual'], df['Predicted'])]
df_metrics=pd.DataFrame.from_dict(metrics)
df_metrics.index = ['Model']

In [ ]:
df_metrics.T

In [ ]:
print('Counts')
print(df_metrics[['TruePositive',
                  'FalseNegative',
                  'FalsePositive',
                  'TrueNegative',]].round(2))
print()
print('Fractions')
print(df_metrics[['Recall',
                  'FalseNegativeRate',
                  'FalsePositiveRate',
                  'TrueNegativeRate',]].round(2))
print()

print(df_metrics[['Precision',
                  'Accuracy',
                  'F1']].round(2))

In [ ]:
print(classification_report(
    df['Actual'],
    df['Predicted'],
    labels=[1,0],
    target_names=['Followup','Normal']))

In [ ]:
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(df['Actual'],df['Predicted_Proba_Class1'])
print('AUC Score: {0:.3f}'.format(roc_score))

In [ ]:
df.head()

In [ ]:
df = df.sort_values('Actual').reset_index(drop=True)
df.head()
df['Actual'].value_counts()

In [ ]:
normal = df[df['Actual']==0]
followup = df[df['Actual']==1]
plt.figure()
plt.scatter(df.index,df['Actual'],label='actual')
plt.scatter(normal.index,normal['Predicted_Proba_Class1'],label='normal')
plt.scatter(followup.index,followup['Predicted_Proba_Class1'],label='followup')
plt.plot([df.index.min(),df.index.max()],[0.5,0.5],color='r')
plt.xlim(left=0)
plt.xlabel('Sample')
plt.ylabel('Predicted Probability')
plt.title('Followup / Normal')
plt.legend(loc=0)
plt.show()

* The model has done a decent job with negative customers
* However, there are lots of positive customers with a probability < 0.5 and the model is classifying them as negative

#### One approach to improve Recall is to lower the threshold so we can identify more positive customers
* Let's say every customer that needs to be followed up has some cost associated with it (say ```$10```)
* We can ignore true negatives as no action is needed with them
* False negatives can be highly costly as it's a missed opportuinty to address some customer concern or an issue raised by the customer
* Let's say that cost is ```$50 ```

#### Finding the optimal cutoff

```txt
$50 * FN(C) + $0 * TN(C) + $10 * FP(C) + $10 * TP(C)
```

FN(C) means that the false negative percentage is a function of the cutoff, C, and similar for TN, FP, and TP.  We need to find the cutoff, C, where the result of the expression is smallest.

A straightforward way to do this, is to simply run a simulation over a large number of possible cutoffs.  We test 100 possible values in the for loop below.

In [ ]:
pd.crosstab(index=df['Actual'],columns=np.where(df['Predicted_Proba_Class1'] > .5, 1, 0))

In [ ]:
cutoffs = np.arange(0.1, .9, 0.01)
costs = []
for c in cutoffs:
    costs.append(np.sum(np.sum(np.array([[0, 10], [50, 10]]) * 
                               pd.crosstab(index=df['Actual'],columns=np.where(df['Predicted_Proba_Class1'] > c, 1, 0)))))

In [ ]:
costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.ylabel('Cost')
plt.xlabel('Cutoff')
plt.show()

print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

In [ ]:
normal = df[df['Actual']==0]
followup = df[df['Actual']==1]
plt.figure()
plt.scatter(df.index,df['Actual'],label='actual')
plt.scatter(normal.index,normal['Predicted_Proba_Class1'],label='normal')
plt.scatter(followup.index,followup['Predicted_Proba_Class1'],label='followup')
plt.plot([df.index.min(),df.index.max()+50],[cutoffs[np.argmin(costs)],cutoffs[np.argmin(costs)]],color='r',linewidth=3)
plt.xlim(left=0)
plt.xlabel('Sample')
plt.ylabel('Predicted Probability')
plt.title('Followup / Normal')
plt.legend(loc=0)
plt.show()

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(df['Actual'], np.where(df['Predicted_Proba_Class1'] > cutoffs[np.argmin(costs)], 1, 0),labels=[1,0])

# Plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Followup','Normal'],
                      title='Confusion Matrix at {0:0.2f}'.format(cutoffs[np.argmin(costs)]), normalize=True)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(df['Actual'], np.where(df['Predicted_Proba_Class1'] > .5, 1, 0),labels=[1,0])

# Plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Followup','Normal'],
                      title='Confusion Matrix at 0.5', normalize=True)

### If you compare the two confusion matrices, we can now identify 79% of the positives compared to the 67% when cutoff was 0.5

### For a classifier, finding the optimal cutoff based on business cost is a great approach